In [1]:
import pandas as pd
import numpy as np
import glob
import sys
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
py_file_location = "/content/drive/MyDrive"
sys.path.append(os.path.abspath(py_file_location))

In [4]:
from function_tools import *

In [5]:
path_files = r"/content/drive/MyDrive/Data_M/1M_Std_clusters/"

In [6]:
# récupérer les clusters 
# glob aide à faire un trie par regex afin d'en choisir les fichiers nettoyés
list_file= glob.glob(f"{path_files}*")

In [7]:
list_file

['/content/drive/MyDrive/Data_M/1M_Std_clusters/som_100',
 '/content/drive/MyDrive/Data_M/1M_Std_clusters/som_225',
 '/content/drive/MyDrive/Data_M/1M_Std_clusters/km_25',
 '/content/drive/MyDrive/Data_M/1M_Std_clusters/km_100',
 '/content/drive/MyDrive/Data_M/1M_Std_clusters/km_50',
 '/content/drive/MyDrive/Data_M/1M_Std_clusters/som_400']

In [8]:
liste_clean , liste_brut = make_liste(list_file)

In [9]:
list_file

['/content/drive/MyDrive/Data_M/1M_Std_clusters/km_25',
 '/content/drive/MyDrive/Data_M/1M_Std_clusters/km_50',
 '/content/drive/MyDrive/Data_M/1M_Std_clusters/km_100',
 '/content/drive/MyDrive/Data_M/1M_Std_clusters/som_100',
 '/content/drive/MyDrive/Data_M/1M_Std_clusters/som_225',
 '/content/drive/MyDrive/Data_M/1M_Std_clusters/som_400']

In [10]:
%%time
clean_cluster= matrice(liste_clean)

CPU times: user 1.62 s, sys: 615 ms, total: 2.23 s
Wall time: 3.52 s


In [11]:
%%time
brut_cluster= matrice(liste_brut)

CPU times: user 3.19 s, sys: 1.12 s, total: 4.31 s
Wall time: 5.75 s


In [14]:
data = make_df(brut_cluster, clean_cluster)

In [ ]:
data[1][0].head()

data_km : 
1.   index 0 : km25
2.   index 1: km50
3.   index 2: km100

data_SOM : 
1.   index 3 : som100
2.   index 4: som225
3.   index 5: som400

In [ ]:
!pip install gensim==4.0
!pip install python-Levenshtein

In [18]:
from gensim.models import KeyedVectors

In [19]:
# récupérer les embeddings entrainés sur nos données
w2v_std = KeyedVectors.load('/content/drive/MyDrive/Data_M/w2v/w2vec_model_d300_1M_Std')

In [20]:
#encode un message
def encode(msg):
    return np.mean([w2v_std[word] for word in msg if word in w2v_std] or [np.zeros(300)], axis = 0)

In [22]:
# Encoder une liste de messages 
def encoded(liste):
  w2v=[]
  for i in range(len(liste)):
    w2v.append(np.array([encode(msg.split()) for msg in liste[i]]))
  return w2v

Encodage des différents set de clusters

In [23]:
len(clean_cluster)

6

In [24]:
def liste_encoded(liste_cluster):
  dataset_encoded=[]
  for i in range(len(liste_cluster)):
    dataset_encoded.append(encoded(liste_cluster[i]))
  return dataset_encoded

In [25]:
dataset_encoded= liste_encoded(clean_cluster)

In [ ]:
len(dataset_encoded[0])

25

Entrainement des différents set de clusters

In [26]:
from sklearn.ensemble import IsolationForest

# definir le modèle ainsi que ses paramètres
if_model=IsolationForest(n_jobs=-1, random_state=42)


Créer une matrice qui stock les différents models d'IF correspondant aux différents clusters

In [27]:
def train_if (dataset):
  models=[[] for _ in range(len(dataset))]
  for i in range(len(dataset)):
    for n in range(len(dataset[i])):
      if_model=IsolationForest(n_jobs=-1, random_state=42)
      models[i].append(if_model.fit(dataset[i][n]))
  return models

Appliquer IF sur les clusters de Km

In [28]:
%%time
models = train_if(dataset_encoded)

Appliquer IF sur les clusters de SOM

Prédictions pour les différents modèles

In [33]:
def pred(dataset,model):
  preds=[[] for _ in range(len(dataset))]
  for i in range(len(dataset)):
    for n in range(len(dataset[i])):
      preds[i].append(model[i][n].predict(dataset[i][n]))
  return preds

In [34]:
pred= pred(dataset_encoded, models)

In [36]:
def add_pred_to_df(dataset,pred):
  for i in range(len(pred)):
    for n in range(len(pred[i])):
      dataset[i][n]['pred']= pred[i][n]

In [37]:
add_pred_to_df(data, pred)

In [38]:
data[0][0]

,tweet,clean_twt,pred
0,J'espère que cette étude sera reprise dans les...,"esper étud repris médi, met mal pseudo argumen...",1
1,Juste macron avec un masque FaceApp\n,macron masqu faceapp\n,1
2,🚨ALERTE INFO:Le président du Conseil scientifi...,alert info président conseil scientif ( inform...,1
3,Ah pas obligé\n,oblig\n,-1
4,"👉Virus MARBURG : Le virus Marburg, ou Marburg ...","virus marburg virus marburg, marburg marburgvi...",1
...,...,...,...
18534,Covid-19 : le confinement des personnes non-va...,covid-19 confin non-vaccin réclam 6 franc 10\n,1
18535,Dix-sept cas de Covid avérés après le repas de...,cas covid aver rep aîn blendecqu vaccin proteg...,1
18536,#lamatinale ces médecins lobbyistes minimisent...,#lamatinal médecin lobbyist minimisent effet s...,1
18537,Je suis scandalisée de l accusation de Maitre ...,scandalis accus maitr jacubowitcz plateau trai...,1


In [39]:
# Trouver les outliers de chaque cluster
# ceux dont l'index est -1
def outliers(df):
  outliers=[]
  for i in range(len(df)):
    outliers.append(df[i]['tweet'].loc[df[i]['pred']==-1])
  return outliers

In [44]:
liste_cluster=["km25","km50","km100","som100","som225","som400"]

In [41]:
def save_outliers(outliers,string):
  for i in range(len(outliers)):
    with open(f"drive/MyDrive/std/"+string+f"/outliers_{i}", 'w') as f:
      for line in outliers[i]:
        f.write(line)

In [42]:
def loop_outliers(data, liste):
  for i in range(len(data)):
    save_outliers(outliers(data[i]),liste[i])

In [45]:
loop_outliers(data, liste_cluster)